In [1]:
#imports
import numpy as np 
import os
import scipy
from scipy.io import loadmat
import pandas as pd
import tensorflow as tf 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, cohen_kappa_score, hamming_loss, zero_one_loss, coverage_error, average_precision_score
from sklearn.model_selection import train_test_split
import random 
import matplotlib.pyplot as plt
from time import strftime, localtime
from sklearn.model_selection import KFold
random.seed = 11

import warnings
warnings.filterwarnings('ignore')

In [2]:
labels = pd.read_csv("/home/karim/Documents/research/MLMLdatasets/DCASEdatasets/urbanSounds/annotations.csv")

In [4]:
labels.columns

Index(['split', 'sensor_id', 'audio_filename', 'annotator_id',
       '1-1_small-sounding-engine_presence',
       '1-2_medium-sounding-engine_presence',
       '1-3_large-sounding-engine_presence',
       '1-X_engine-of-uncertain-size_presence', '2-1_rock-drill_presence',
       '2-2_jackhammer_presence', '2-3_hoe-ram_presence',
       '2-4_pile-driver_presence',
       '2-X_other-unknown-impact-machinery_presence',
       '3-1_non-machinery-impact_presence', '4-1_chainsaw_presence',
       '4-2_small-medium-rotating-saw_presence',
       '4-3_large-rotating-saw_presence',
       '4-X_other-unknown-powered-saw_presence', '5-1_car-horn_presence',
       '5-2_car-alarm_presence', '5-3_siren_presence',
       '5-4_reverse-beeper_presence',
       '5-X_other-unknown-alert-signal_presence',
       '6-1_stationary-music_presence', '6-2_mobile-music_presence',
       '6-3_ice-cream-truck_presence',
       '6-X_music-from-uncertain-source_presence',
       '7-1_person-or-small-group-talking_p

In [67]:
major_labels = labels[['audio_filename', '1_engine_presence',
       '2_machinery-impact_presence', '3_non-machinery-impact_presence',
       '4_powered-saw_presence', '5_alert-signal_presence', '6_music_presence',
       '7_human-voice_presence', '8_dog_presence']]

In [68]:
# replacing the missing values of (-1) with nan
major_labels = major_labels.replace(-1,np.nan)
# Compute the mean of the annotations, we wil include only ones that fulle agree, i.e. mean == 1
filtered_major_labels = major_labels.groupby('audio_filename').mean()
filtered_major_labels[filtered_major_labels < 1] = 0 

In [70]:
# the number of samples with labels that are don't fully agree  is quite high! 
# 2169 are not fully agreed on
print('number of samples with no labels due to no full agreement between annotators is',
      (filtered_major_labels.sum(axis =1) == 0).sum())

number of samples with no labels due to no full agreement between annotators is 2169


In [89]:
# We replace the hard limit to at least half the annotators agree on it, i.e. mean = 0.5
filtered_major_labels = major_labels.groupby('audio_filename').mean()
filtered_major_labels[filtered_major_labels < 0.5] = 0 
# 787 are not half agreed on, we will ignore them for the sake of having confidence in the labels
(filtered_major_labels.sum(axis =1) == 0).sum()
print('number of samples with no labels due to less than 50% agreement is',
      (filtered_major_labels.sum(axis =1) == 0).sum())
print('number of samples left after sampling is',
      (filtered_major_labels.sum(axis =1) != 0).sum())

number of samples with no labels due to less than 50% agreement is 787
number of samples left after sampling is 2281


In [83]:
filtered_major_labels = filtered_major_labels[filtered_major_labels.sum(axis =1) != 0]
# replace values between 0.5 and 1 with 1
filtered_major_labels[filtered_major_labels != 0] = 1 
print('Average nmber of labels per sample {:.2f}'.format(filtered_major_labels.sum(axis=1).mean()))
print('So lowww')

Average nmber of labels per sample 1.25
So lowww
